In [1]:
import os
import pandas as pd
import datetime as dt
import collections

### Recherche des fichiers Excel dans le repertoire et extraction de la date dans le titre

In [2]:
folder= r"C:\Users\sgasmi\Desktop\DATA HOME 2020"   

In [3]:
sources = pd.DataFrame(columns=["Fichier", "Folder", "File Date", "Conservé"])
i = 0

for root, dirs, files in os.walk(folder):
    for filename in files:
        sources.at[i,"Fichier"] = filename
        sources.at[i, "Folder"] = folder + "\\" + filename
        sources.at[i, "File Date"] = filename[10:][:8]
        print(sources.at[i, "File Date"])
        try:
            sources.at[i, "File Date"] = dt.datetime.strptime(sources.at[i, "File Date"], '%d%m%Y').date()
        except:
            sources.at[i, "Conservé"] = "False"
        i+=1

29022020
30042020
30062020
30092020
30112020
31012020
31032020
31052020
31072020
31082020
31102020
31122019
E 311020
minative


In [4]:
source = sources.loc[sources["Conservé"]!="False"].reset_index(drop=True)
source = source.sort_values(["File Date"])
source["custom"]=1

### Fusion des dataframes

In [5]:
merge = pd.DataFrame()

for i, val in enumerate(source.iterrows()):
    target = folder + "\\" + source.at[i,"Fichier"]
    res = pd.read_excel(target, sheet_name="Données")
    res["Date"] =  source.at[i, "File Date"]
    print(source.at[i,"Fichier"])
    # fusion des dataframe
    if i==0:
        merge=res
    else:
        merge = merge.append(res, ignore_index=True)

DATA_HOME 29022020.xlsx
DATA_HOME 30042020.xlsx
DATA_HOME 30062020.xlsx
DATA_HOME 30092020.xlsx
DATA_HOME 30112020.xlsx
DATA_HOME 31012020.xlsx
DATA_HOME 31032020.xlsx
DATA_HOME 31052020.xlsx
DATA_HOME 31072020.xlsx
DATA_HOME 31082020.xlsx
DATA_HOME 31102020.xlsx
DATA_HOME 31122019.xlsx


In [6]:
colonnes = ["local payroll ID","Date", "Org. Payroll", "Legal entity","Sex", "Category", "Home Country",  "Last name", "First name"]
merge_exp=merge[colonnes]

# Non conservation lignes vides
merge_exp = merge_exp.loc[merge_exp["local payroll ID"].isna()==False]
merge_exp = merge_exp.reset_index(drop=True)
merge_exp["Headcount"]=1
colonne = colonnes.append("Headcount")

### Suppression des doublons

In [7]:
merge_exp= merge_exp.drop_duplicates(["local payroll ID", "Date"])
merge_exp = merge_exp.reset_index(drop=True)

### Création d'une table de fait dégradée

In [8]:
merge_fact = pd.DataFrame()
merge_fact["local payroll ID"] = merge_exp["local payroll ID"].drop_duplicates()
merge_fact["custom"] = 1
merge_fact = merge_fact.merge(source[["custom", "File Date"]], how="outer", on=["custom"])
merge_fact= merge_fact.drop(["custom"], axis=1)
merge_fact.columns= ["local payroll ID", "Date"]

In [9]:
merge_fact = merge_fact.merge(merge_exp[colonnes], how="left", on=["local payroll ID", "Date"])
merge_fact.loc[merge_fact["Headcount"]!=1,"Headcount"]=0

### Pour les salariés sortis, conserver la dernière situation

In [10]:
dmin = source["File Date"].min()
col_control = colonnes[3]

for i, val in enumerate(merge_fact.iterrows()):
    merge_fact.at[i, col_control]
    
    if str(merge_fact.at[i, col_control]) =="nan" and merge_fact.at[i, "Date"]!=dmin:
        if merge_fact.at[i,"local payroll ID"]==merge_fact.at[i-1,"local payroll ID"]:
            for col in colonnes[2:-1]:                
                merge_fact.at[i, col] =  merge_fact.at[i-1, col]
        
        

### Ajout indicateur sorti

In [11]:
merge_fact["Leave"] = 0
for i, val in enumerate(merge_fact.iterrows()):
    if merge_fact.at[i, "Date"]!=dmin:
        if (merge_fact.at[i,"local payroll ID"]==merge_fact.at[i-1,"local payroll ID"]) and (merge_fact.at[i-1,"Headcount"]==1 and merge_fact.at[i,"Headcount"]==0):
            merge_fact.at[i,"Leave"]=1

### Ajout indicateur entrée

In [ ]:
merge_fact["Entree"] = 0
for i, val in enumerate(merge_fact.iterrows()):
    if merge_fact.at[i, "Date"]!=dmin:
        if (merge_fact.at[i,"local payroll ID"]==merge_fact.at[i-1,"local payroll ID"]) and (merge_fact.at[i-1,"Headcount"]==0 and merge_fact.at[i,"Headcount"]==1):
            merge_fact.at[i,"Entree"]=1

### Suppression nan pour colonne[2]

In [13]:
merge_fact[colonnes[3]] = merge_fact[colonnes[3]].fillna("")

### Export Excel

In [14]:
with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\Anomalie UO.xlsx') as writer:  
        anomalie_niveau.to_excel(writer, sheet_name='Anomalies_niveau', index=False)
        anomalie_RH.to_excel(writer, sheet_name='Anomalies_RH', index=False)
        anomalie_Manager.to_excel(writer, sheet_name='Anomalie_Manager', index=False)